In [1]:
from datasets import load_dataset
from bs4 import BeautifulSoup
import re
import pandas as pd
import warnings

In [14]:
def cleanup_comment(comment):
    text = BeautifulSoup(comment).get_text(separator = '\n', strip = True)
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    # text = re.sub(r'>>[0-9]+', '', text, flags=re.MULTILINE)
    return text.strip()

def process_thread(examples):
    comments = []
    for thread in examples['text']:
        lines = [line for line in thread.split('\n') if line]
        topic = lines[0].split('<>')[4]
        topic = BeautifulSoup(topic).get_text(strip = True)
        topic = re.sub(r' ?\[.+$', '', topic)
        topic = re.sub(r'★', '', topic)
        topic = re.sub(r'[0-9]+ ?$', '', topic)
        topic = re.sub(r'[０１２３４５６７８９]+ ?$', '', topic)
        comments += [(topic, cleanup_comment(line.split('<>')[3])) for line in lines][1:999]
    return {"topic": [topic for (topic, _) in comments], "comment": [comment for (_, comment) in comments]}

DatasetDict({
    train: Dataset({
        features: ['topic', 'comment'],
        num_rows: 1121290
    })
    test: Dataset({
        features: ['topic', 'comment'],
        num_rows: 129031
    })
})

In [16]:
dataset = load_dataset("text", data_dir="scraped", sample_by='document')['train'].train_test_split(test_size=0.1)
dataset = dataset.map(process_thread, batched=True, remove_columns=['text'], batch_size=1, num_proc=16)
dataset = dataset.filter(lambda example: ">>" not in example["comment"])
dataset = dataset.filter(lambda example: len(example["comment"]) < 50)
dataset = dataset.shuffle()

Resolving data files:   0%|          | 0/4068 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=16):   0%|          | 0/3661 [00:00<?, ? examples/s]

/home/peryaudo/mambaforge/envs/peryagpt/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/home/peryaudo/mambaforge/envs/peryagpt/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/home/peryaudo/mambaforge/envs/peryagpt/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/home/peryaudo/mambaforge/envs/peryagpt/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/home/peryaudo/mambaforge/envs/peryagpt/lib/python3.11/site-packages/datasets/table.py:1387: FutureWarning: 

...

Filter:   0%|          | 0/2296088 [00:00<?, ? examples/s]

Filter:   0%|          | 0/262970 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1533945 [00:00<?, ? examples/s]

Filter:   0%|          | 0/175846 [00:00<?, ? examples/s]

AttributeError: 'DatasetDict' object has no attribute 'select'

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)
train_dataset = dataset['train']
train_dataset = train_dataset.select(range(int(len(train_dataset) * 0.1)))
train_dataset.to_pandas().head(1000)